In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [2]:
data, info = tfds.load(name='deep_weeds', as_supervised=True, with_info=True)
NUM_CLASSES = info.features['label'].num_classes
DATASET_SIZE = info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Shuffling deep_weeds-train.tfrecord...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Dataset deep_weeds downloaded and prepared to /home/jupyter/tensorflow_datasets/deep_weeds/3.0.0. Subsequent calls will reuse this data.


2022-01-25 10:25:08.265776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-01-25 10:25:08.265907: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-25 10:25:08.265942: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vm-abb6abc9-ad72-4c89-a68d-2d22d42d88da): /proc/driver/nvidia/version does not exist
2022-01-25 10:25:08.370381: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  

In [3]:
def preprocess_data(image, label):
  image = tf.image.resize(image, (300,300))
  return tf.cast(image, tf.float32) / 255., label

In [4]:
# Create train/validation splits

# Shuffle dataset
dataset = data['train'].shuffle(1000)

train_split = 0.8
val_split = 0.2
train_size = int(train_split * DATASET_SIZE)
val_size = int(val_split * DATASET_SIZE)

train_data = dataset.take(train_size)
train_data  = train_data.map(preprocess_data)
train_data  = train_data.batch(64)

validation_data = dataset.skip(train_size)
validation_data  = validation_data.map(preprocess_data)
validation_data  = validation_data.batch(64)

In [6]:
feature_extractor_model = "inception_v3"

In [7]:
tf_hub_uri = f"https://tfhub.dev/google/imagenet/{feature_extractor_model}/feature_vector/5"


In [8]:
feature_extractor_layer = hub.KerasLayer(
    tf_hub_uri,
    trainable=False)

In [9]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(units=NUM_CLASSES)
])

In [10]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(train_data, validation_data=validation_data, epochs=20)

Epoch 1/20
 19/219 [=>............................] - ETA: 22:30 - loss: 1.5948 - acc: 0.5321

KeyboardInterrupt: 